# Teradata to BigQuery SQL Translation

## Introduction

Both BigQuery and Teradata Database conform to the [ANSI/ISO SQL:2011](https://wikipedia.org/wiki/SQL:2011) standard. In addition, Teradata has created some extensions to the SQL standard to enable Teradata-specific functionalities.

In contrast, BigQuery does not support these proprietary extensions. Therefore, some of your queries might need to be refactored during migration from Teradata to BigQuery. Having queries that only use the ANSI/ISO SQL standard that's supported by BigQuery has the added benefit that it helps ensure portability and helps your queries be agnostic to the underlying data warehouse.

This notebook addresses some of the challenges you might encounter while migrating SQL queries from Teradata to BigQuery. It explains when these translations should be applied in the context of an end-to-end staged migration.

* basic replace stuff?
* qualify
* sub selects?
* Multiset?

## Teradata SQL differences

This notebook discusses notable differences between Teradata SQL and the BigQuery standard SQL, and some strategies for translating between the two dialects. The list of differences presented in this notebook is not exhaustive. For additional information, see the [Teradata-to-BigQuery SQL translation reference](https://cloud.google.com/solutions/migration/dw2bq/td2bq/td-bq-sql-translation-reference-tables).

## Data Types

BigQuery supports a more concise set of data types than
Teradata, with groups of Teradata types mapping into a single standard SQL data
type. For instance:

-   `INTEGER`, `SMALLINT`, `BYTEINT`, and `BIGINT` all map to `INT64`.
-   `CLOB`, `JSON`, `XML`, `UDT` and other types that contain large
    character fields map to `STRING`.
-   `BLOB`, `BYTE`, and `VARBYTE` types that contain binary information map
    to `BYTES`.

For dates, the main types (`DATE`, `TIME`, and `TIMESTAMP`) are equivalent in
Teradata and BigQuery. However, other specialized date types from
Teradata need to be mapped, such as the following:

-   `TIME_WITH_TIME_ZONE` to `TIME`.
-   `TIMESTAMP_WITH_TIME_ZONE` to `TIMESTAMP`.
-   `INTERVAL_HOUR`, `INTERVAL_MINUTE`, and other `INTERVAL_*` types map to
    `INT64` in BigQuery.
-   `PERIOD(DATE)`,` PERIOD(TIME)`, and other` PERIOD(*)` types map to `STRING`.

[Multi-dimensional arrays](https://docs.teradata.com/reader/S0Fw2AVH8ff3MDA0wDOHlQ/D3QuBsLccP9JObIH8f4yJA)
are not directly supported in BigQuery. Instead, you create an
[array of structs](/bigquery/docs/reference/standard-sql/arrays#building_arrays_of_arrays),
with each struct containing a field of type `ARRAY`.

## Data Types - Exercise

In this exercise, you will examine several of the TIMESTAMP and TIME functions and data types available to you. You will be using a public BigQuery dataset that contains rental records from the London bike share program

Use the `bq` command line tool to examine the schema of the table.

`bq head` or using the `Preview` tab in the BigQuery UI are much more efficient than a `SELECT * LIMIT 1` as this triggers a whole table scan.

In [45]:
!bq head -n 5  --selected_fields rental_id,duration,bike_id,end_date,end_station_id,start_date,start_station_id bigquery-public-data:london_bicycles.cycle_hire

+-----------+----------+---------+---------------------+----------------+---------------------+------------------+
| rental_id | duration | bike_id |      end_date       | end_station_id |     start_date      | start_station_id |
+-----------+----------+---------+---------------------+----------------+---------------------+------------------+
|  47469109 |     3180 |    7054 | 2015-09-03 12:45:00 |            111 | 2015-09-03 11:52:00 |              300 |
|  46915469 |     7380 |    3792 | 2015-08-16 11:59:00 |            407 | 2015-08-16 09:56:00 |              407 |
|  65899423 |     2040 |    3038 | 2017-06-09 18:30:00 |            165 | 2017-06-09 17:56:00 |              579 |
|  64280726 |     2280 |   10868 | 2017-04-22 10:14:00 |            553 | 2017-04-22 09:36:00 |              519 |
|  59235489 |     2340 |    7183 | 2016-10-09 04:31:00 |            100 | 2016-10-09 03:52:00 |              612 |
+-----------+----------+---------+---------------------+----------------+-------

We can similarily see table level data, such as number of rows and the schema of the table. Notice the `TIMESTAMP` fields.

In [62]:
!bq show bigquery-public-data:london_bicycles.cycle_hire

Table bigquery-public-data:london_bicycles.cycle_hire

   Last modified                      Schema                     Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Labels  
 ----------------- -------------------------------------------- ------------ ------------- ------------ ------------------- ------------------ -------- 
  08 May 11:21:52   |- rental_id: integer (required)             24369201     2784394803                                                                
                    |- duration: integer                                                                                                                
                    |- bike_id: integer                                                                                                                 
                    |- end_date: timestamp                                                                                                              
                    |- end_

Run a query to return the most recent 5 rentals by end_date:

In [63]:
%%bigquery

SELECT
  rental_id,
  duration,
  bike_id,
  end_date,
  end_station_id,
  end_station_name,
  start_date,
  start_station_id,
  start_station_name
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
ORDER BY
  end_date DESC
LIMIT
  5

,rental_id,duration,bike_id,end_date,end_station_id,end_station_name,start_date,start_station_id,start_station_name
0,66036945,56220,13042,2017-06-14 03:25:00+00:00,501,"Cephas Street, Bethnal Green",2017-06-13 11:48:00+00:00,501,"Cephas Street, Bethnal Green"
1,66036927,56280,6631,2017-06-14 03:25:00+00:00,501,"Cephas Street, Bethnal Green",2017-06-13 11:47:00+00:00,501,"Cephas Street, Bethnal Green"
2,66063574,15720,6088,2017-06-14 02:34:00+00:00,556,"Heron Quays DLR, Canary Wharf",2017-06-13 22:12:00+00:00,477,"Spindrift Avenue, Millwall"
3,66063566,15780,4726,2017-06-14 02:34:00+00:00,556,"Heron Quays DLR, Canary Wharf",2017-06-13 22:11:00+00:00,477,"Spindrift Avenue, Millwall"
4,66037030,52440,3686,2017-06-14 02:26:00+00:00,523,"Langdon Park, Poplar",2017-06-13 11:52:00+00:00,501,"Cephas Street, Bethnal Green"


__#TODO(you):__ Modify this query to print the `end_date` and `start_date` fields in UNIX seconds as well.

[Hint: Use UNIX_SECONDS().](https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions#unix_seconds)

In [64]:
%%bigquery

SELECT
  rental_id,
  duration,
  bike_id,
  end_date,
  UNIX_SECONDS(end_date) AS end_date_unix,
  end_station_id,
  end_station_name,
  start_date,
  UNIX_SECONDS(start_date) AS start_date_unix,
  start_station_id,
  start_station_name
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
ORDER BY
  end_date DESC
LIMIT
  5

,rental_id,duration,bike_id,end_date,end_date_unix,end_station_id,end_station_name,start_date,start_date_unix,start_station_id,start_station_name
0,66036927,56280,6631,2017-06-14 03:25:00+00:00,1497410700,501,"Cephas Street, Bethnal Green",2017-06-13 11:47:00+00:00,1497354420,501,"Cephas Street, Bethnal Green"
1,66036945,56220,13042,2017-06-14 03:25:00+00:00,1497410700,501,"Cephas Street, Bethnal Green",2017-06-13 11:48:00+00:00,1497354480,501,"Cephas Street, Bethnal Green"
2,66063574,15720,6088,2017-06-14 02:34:00+00:00,1497407640,556,"Heron Quays DLR, Canary Wharf",2017-06-13 22:12:00+00:00,1497391920,477,"Spindrift Avenue, Millwall"
3,66063566,15780,4726,2017-06-14 02:34:00+00:00,1497407640,556,"Heron Quays DLR, Canary Wharf",2017-06-13 22:11:00+00:00,1497391860,477,"Spindrift Avenue, Millwall"
4,66037030,52440,3686,2017-06-14 02:26:00+00:00,1497407160,523,"Langdon Park, Poplar",2017-06-13 11:52:00+00:00,1497354720,501,"Cephas Street, Bethnal Green"


__#TODO(you):__ Modify this query to print the time from the `end_date` and `start_date` fields in formatted PST timezone.

[Hint: Use EXTRACT( ... AT TIME ZONE ... ).](https://cloud.google.com/bigquery/docs/reference/standard-sql/timestamp_functions#extract)

In [65]:
%%bigquery

SELECT
  rental_id,
  duration,
  bike_id,
  end_date,
  EXTRACT(TIME FROM end_date AT TIME ZONE "America/Los_Angeles") AS end_time_california,
  end_station_id,
  end_station_name,
  start_date,
  EXTRACT(TIME FROM start_date AT TIME ZONE "America/Los_Angeles") AS start_time_california,
  start_station_id,
  start_station_name
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
ORDER BY
  end_date DESC
LIMIT
  5

,rental_id,duration,bike_id,end_date,end_time_california,end_station_id,end_station_name,start_date,start_time_california,start_station_id,start_station_name
0,66036927,56280,6631,2017-06-14 03:25:00+00:00,20:25:00,501,"Cephas Street, Bethnal Green",2017-06-13 11:47:00+00:00,04:47:00,501,"Cephas Street, Bethnal Green"
1,66036945,56220,13042,2017-06-14 03:25:00+00:00,20:25:00,501,"Cephas Street, Bethnal Green",2017-06-13 11:48:00+00:00,04:48:00,501,"Cephas Street, Bethnal Green"
2,66063574,15720,6088,2017-06-14 02:34:00+00:00,19:34:00,556,"Heron Quays DLR, Canary Wharf",2017-06-13 22:12:00+00:00,15:12:00,477,"Spindrift Avenue, Millwall"
3,66063566,15780,4726,2017-06-14 02:34:00+00:00,19:34:00,556,"Heron Quays DLR, Canary Wharf",2017-06-13 22:11:00+00:00,15:11:00,477,"Spindrift Avenue, Millwall"
4,66037030,52440,3686,2017-06-14 02:26:00+00:00,19:26:00,523,"Langdon Park, Poplar",2017-06-13 11:52:00+00:00,04:52:00,501,"Cephas Street, Bethnal Green"


## The SELECT Statement


The syntax of the `SELECT` statement is generally compatible between Teradata and
BigQuery. This section notes differences that often must be
addressed during migration.

### Identifiers

BigQuery lets you use the following as
[identifiers](https://cloud.google.com/bigquery/docs/reference/standard-sql/lexical#identifiers): projects;datasets; tables or views; columns.

As a serverless product, BigQuery does not have a concept of a
cluster or environment or fixed endpoint, therefore the project specifies the dataset's
[resource hierarchy](https://cloud.google.com/resource-manager/docs/cloud-platform-resource-hierarchy).


In a `SELECT` statement in Teradata, fully qualified column names can be used.
BigQuery always references column names from tables or aliases,
and never from projects or datasets.

For example, here are some options to address identifiers in BigQuery.

Columns implicitly inferred from the table:

```sql
SELECT
 c
FROM
 project.dataset.table
```






Or by using an explicit table reference:

```sql
SELECT
 table.c
FROM
 project.dataset.table
```

Or by using an explicit table alias:

```sql
SELECT
 t.c
FROM
 project.dataset.table t
```

__#TODO(you):__ Run the following queries showing the different indentifier options.

In [75]:
%%bigquery

SELECT
  rental_id,
  duration,
  bike_id
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
LIMIT
  1

,rental_id,duration,bike_id
0,47469109,3180,7054


In [76]:
%%bigquery

SELECT
  cycle_hire.rental_id,
  cycle_hire.duration,
  cycle_hire.bike_id
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
LIMIT
  1

,rental_id,duration,bike_id
0,47469109,3180,7054


In [77]:
%%bigquery

SELECT
  r.rental_id,
  r.duration,
  r.bike_id
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire r
LIMIT
  1

,rental_id,duration,bike_id
0,58406630,1020,8168


### Alias references

In a `SELECT` statement in Teradata, aliases can be defined and referenced
within the same query. For instance, in the following snippet, `flag` is defined
as a column alias, and then immediately referred to in the enclosed `CASE`
statement.

```sql
SELECT
 F AS flag,
 CASE WHEN flag = 1 THEN ...
```




In standard SQL, references between columns *within the same query* are not
allowed. To translate, you move the logic into a nested query:

```sql
SELECT
 q.*,
 CASE WHEN q.flag = 1 THEN ...
FROM (
 SELECT
   F AS flag,
   ...
) AS q
```

The sample placeholder `F` could itself be a nested query that returns a single
column.


__#TODO(you):__ Run the following query, notice the syntax error, and rewrite it with a nested query to conform to standard SQL.

_Note:_ You could just move the EXTRACT() function but for the purposes of the exercise use a nested query.

In [87]:
%%bigquery
-- Query should fail

SELECT
  rental_id,
  duration,
  bike_id,
  start_date,
  EXTRACT(HOUR FROM start_date) AS start_hour,
  CASE
    WHEN start_hour <= 12 THEN TRUE
  ELSE FALSE
END
  AS morning_ride
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
LIMIT
  1

Executing query with job ID: 13132f61-d9c0-4896-8df6-fec241e38898
Query executing: 0.43s


ERROR:
 400 Unrecognized name: start_hour; Did you mean start_date? at [9:10]

(job ID: 13132f61-d9c0-4896-8df6-fec241e38898)

            -----Query Job SQL Follows-----             

    |    .    |    .    |    .    |    .    |    .    |
   1:-- Query should fail
   2:SELECT
   3:  rental_id,
   4:  duration,
   5:  bike_id,
   6:  start_date,
   7:  EXTRACT(HOUR FROM start_date) AS start_hour,
   8:  CASE
   9:    WHEN start_hour <= 12 THEN TRUE
  10:  ELSE FALSE
  11:END
  12:  AS morning_ride
  13:FROM
  14:  `bigquery-public-data`.london_bicycles.cycle_hire
  15:LIMIT
  16:  1
    |    .    |    .    |    .    |    .    |    .    |


In [88]:
%%bigquery
-- Query should succeed

SELECT
  *,
  CASE
    WHEN start_hour <= 12 THEN TRUE
  ELSE FALSE
END
  AS morning_ride
FROM (
  SELECT
    rental_id,
    duration,
    bike_id,
    start_date,
    EXTRACT(HOUR
    FROM
      start_date) AS start_hour
  FROM
    `bigquery-public-data`.london_bicycles.cycle_hire ) AS rentals
LIMIT
  1

,rental_id,duration,bike_id,start_date,start_hour,morning_ride
0,58406630,1020,8168,2016-09-15 10:09:00+00:00,10,True


### Filtering with LIKE

In Teradata, the `LIKE ANY` operator is used to filter the results to a given
set of possible options. For example:

```sql
SELECT*
FROM t1
WHERE a LIKE ANY ('string1', 'string2')
```

To translate statements that have this operator to standard SQL, you can split
the list after `ANY` into several `OR` predicates:

```sql
SELECT*
FROM t1
WHERE a LIKE 'string1' OR a LIKE 'string2'
```

__#TODO(you):__ Rewrite this query with OR predicate so that it succeeds.

In [92]:
%%bigquery
-- Query should fail

SELECT
  rental_id,
  duration,
  bike_id,
  start_date,
  start_station_name
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
WHERE
  start_station_name LIKE ANY ('%Hyde Park%', '%Soho%')
LIMIT
  5

Executing query with job ID: c6bb6e3b-06c1-4456-99dd-53214a1f9b69
Query executing: 0.38s


ERROR:
 400 Syntax error: Unexpected keyword ANY at [11:27]

(job ID: c6bb6e3b-06c1-4456-99dd-53214a1f9b69)

              -----Query Job SQL Follows-----               

    |    .    |    .    |    .    |    .    |    .    |
   1:-- Query should fail
   2:SELECT
   3:  rental_id,
   4:  duration,
   5:  bike_id,
   6:  start_date,
   7:  start_station_name
   8:FROM
   9:  `bigquery-public-data`.london_bicycles.cycle_hire
  10:WHERE
  11:  start_station_name LIKE ANY ('%Hyde Park%', '%Soho%')
  12:LIMIT
  13:  5
    |    .    |    .    |    .    |    .    |    .    |


In [93]:
%%bigquery
-- Query should fail

SELECT
  rental_id,
  duration,
  bike_id,
  start_date,
  start_station_name
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
WHERE
  start_station_name LIKE '%Hyde Park%'
  OR start_station_name LIKE '%Soho%'
LIMIT
  5

,rental_id,duration,bike_id,start_date,start_station_name
0,47469109,3180,7054,2015-09-03 11:52:00+00:00,"Serpentine Car Park, Hyde Park"
1,46915469,7380,3792,2015-08-16 09:56:00+00:00,"Speakers' Corner 1, Hyde Park"
2,47150505,2400,6748,2015-08-22 16:35:00+00:00,"Triangle Car Park, Hyde Park"
3,57635395,6060,1532,2016-08-25 16:04:00+00:00,"Broadwick Street, Soho"
4,63440296,2280,8998,2017-03-27 17:14:00+00:00,"Park Lane , Hyde Park"


### The QUALIFY clause



Teradata's
[QUALIFY](https://docs.teradata.com/reader/2_MC9vCtAJRlKle2Rpb0mA/19NnI91neorAi7LX6SJXBw) clause is a conditional clause in the `SELECT` statement that filters results of a previously computed, ordered [analytic function](https://cloud.google.com/bigquery/docs/reference/standard-sql/analytic-function-concepts) according to user‑specified search conditions. Its syntax consists of the `QUALIFY` clause followed by the analytic function, such as [`ROW_NUMBER`](https://docs.teradata.com/reader/kmuOwjp1zEYg98JsB8fu_A/8AEiTSe3nkHWox93XxcLrg) or [`RANK`](https://docs.teradata.com/reader/kmuOwjp1zEYg98JsB8fu_A/8Ex9CS5XErnUTmh7zcrOPg), and the values you want to find:

```sql
SELECT a, b
FROM t1
QUALIFY ROW_NUMBER() OVER (PARTITION BY a ORDER BY b) = 1
```



Teradata users commonly use this function as a shorthand way to rank and
return results without the need for an additional subquery.

The `QUALIFY` clause is translated to BigQuery by adding a
`WHERE` condition to an enclosing query:

```sql
SELECT a, b
FROM (
 SELECT a, b,
 ROW_NUMBER() OVER (PARTITION BY A ORDER BY B) row_num
 FROM t1
) WHERE row_num = 1
```

__#TODO(you):__ Rewrite this query such that it succeeds without a QUALIFY clause

This query is returning the very first completed rental for each unique `bike_id`, ordered by end_date.

In [101]:
%%bigquery
-- Query should fail

SELECT
  rental_id,
  duration,
  bike_id,
  end_date
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire
  QUALIFY ROW_NUMBER() OVER (PARTITION BY bike_id ORDER BY end_date ASC) = 1
LIMIT
  5

Executing query with job ID: 32d93aec-454b-425f-956e-b198b4ba939a
Query executing: 0.22s


ERROR:
 400 Syntax error: Expected end of input but got identifier "ROW_NUMBER" at [10:11]

(job ID: 32d93aec-454b-425f-956e-b198b4ba939a)

                         -----Query Job SQL Follows-----                         

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:-- Query should fail
   2:
   3:SELECT
   4:  rental_id,
   5:  duration,
   6:  bike_id,
   7:  end_date
   8:FROM
   9:  `bigquery-public-data`.london_bicycles.cycle_hire
  10:  QUALIFY ROW_NUMBER() OVER (PARTITION BY bike_id ORDER BY end_date ASC) = 1
  11:LIMIT
  12:  5
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


In [104]:
%%bigquery
-- Query should succeed

SELECT
  rental_id,
  duration,
  bike_id,
  end_date
FROM (
  SELECT
    rental_id,
    duration,
    bike_id,
    end_date,
    ROW_NUMBER() OVER (PARTITION BY bike_id ORDER BY end_date ASC) rental_num
  FROM
    `bigquery-public-data`.london_bicycles.cycle_hire )
WHERE
  rental_num = 1
LIMIT
  5

,rental_id,duration,bike_id,end_date
0,40352695,660,285,2015-01-04 15:40:00+00:00
1,40384822,360,351,2015-01-06 09:12:00+00:00
2,40570295,300,813,2015-01-15 09:07:00+00:00
3,40347059,1860,983,2015-01-04 04:21:00+00:00
4,50501327,1920,1177,2015-12-24 22:03:00+00:00


### Notes on Scalable Analytic and Aggregate Functions


Many of the Analytic Functions and Aggregate Functions in BigQuery have been implemented in a distributed, scalable manner, meaning it is now harder to overload a single worker. If you have highly skewed data (for example a single `bike_id` accounts for 95% of rides) or you are sorting a very large dataset, this used to be processed on a single BigQuery worker.

That said, it is still important to utilize BigQuery best-pratcies wherever possible. For example filtering early and often and applying `LIMIT` clauses on aggregate functions like `ARRAY_AGG()`.

The 'latest record' use-case has a particularly fast implementation using `ARRAY_AGG(.... LIMIT 1)[offset(0)]` which allows can run more efficiently because the `ORDER BY` is allowed to drop everything except the top record on each `GROUP BY`

In this example query, we are no longer grouping by `bike_id`, so we are asking BigQuery to sort the entire dataset _and_ assign a row_number to all 24 million rows  before only picking the first one:

In [112]:
%%bigquery --verbose
-- Query should succeed, but will take a bit

SELECT
  rental_id,
  duration,
  bike_id,
  end_date
FROM (
  SELECT
    rental_id,
    duration,
    bike_id,
    end_date,
    
    -- NOTE: we removed the 'PARTITION BY bike_id' clause
    ROW_NUMBER() OVER (ORDER BY end_date ASC) rental_num
    
  FROM
    `bigquery-public-data`.london_bicycles.cycle_hire )
WHERE
  rental_num = 1

Executing query with job ID: 4eb1698e-5e8b-44f4-b687-99a622777e3b
Query executing: 15.22s
Query complete after 15.88s


,rental_id,duration,bike_id,end_date
0,40346512,120,7793,2015-01-04 00:03:00+00:00


We can apply the `ARRAY_AGG(.... LIMIT 1)[offset(0)]` trick to this query to speed it up greatly.

In [113]:
%%bigquery --verbose
-- Query should succeed more quickly

SELECT
  rental.*
FROM (
  SELECT
    ARRAY_AGG( rentals
    ORDER BY rentals.end_date ASC LIMIT 1)[OFFSET(0)] rental
  FROM (
    SELECT
      rental_id,
      duration,
      bike_id,
      end_date
    FROM
      `bigquery-public-data`.london_bicycles.cycle_hire) rentals )
LIMIT
  5

Executing query with job ID: 99b4a883-d739-4112-8f5f-57c5e336265a
Query executing: 2.21s
Query complete after 2.85s


,rental_id,duration,bike_id,end_date
0,40346512,120,7793,2015-01-04 00:03:00+00:00


The first rule of BigQuery optimization is if your query runs in an acceptable amount of time and with acceptable resources, don't fix it! BigQuery has lots of intelligent (and brute-force) tricks under the hood to optimize your query for you.

For example, applying this ARRAY_AGG() trick to the original query where we had a `GROUP BY bike_id` class will greatly slow it down, mostly because this dataset is too small to benefit from this trick.

__Bonus:__ Try this trick on the previous query and see if it's faster or slower

## Data Definition Language (DDL)

## Data Manipulation Language (DML)

The [Data Manipulation Language (DML)](https://wikipedia.org/wiki/Data_manipulation_language) is used to list, add, delete, and modify data in a database. It includes the
`SELECT`, `INSERT`, `DELETE`, and `UPDATE` statements.

While the basic forms of these statements are the same between Teradata SQL and
standard SQL, Teradata includes additional, non-standard clauses and special
statement constructs that you need to convert when you migrate. The following
sections present a non-exhaustive list of the most common statements, the main
differences, and the recommended translations.

### The INSERT statement

BigQuery is an enterprise data warehouse that focuses on Online
Analytical Processing (OLAP). Using point-specific DML statements, such as
executing a script with many `INSERT` statements, is an attempt to treat
BigQuery like an Online Transaction Processing (OLTP) system,
which is not a correct approach.



BigQuery DML statements are intended for bulk updates, therefore
each DML statement that modifies data
[initiates an implicit transaction](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-manipulation-language#limitations).
You should group your DML statements whenever possible to avoid unnecessary
transaction overhead.

As an example, if you have the following set of statements from Teradata,
running them as is in BigQuery is an anti-pattern:

```sql
INSERT INTO t1 (...) VALUES (...);
INSERT INTO t1 (...) VALUES (...);
```

You can translate the previous script into a single `INSERT` statement, which
performs a bulk operation instead:

```sql
INSERT INTO t1 VALUES (...), (...)
```



A typical scenario where a large number of `INSERT` statements is used is when
you create a new table from an existing  table. In BigQuery,
instead of using multiple `INSERT` statements, create a new table and insert all
the rows in one operation using the
[`CREATE TABLE ... AS SELECT`](https://cloud.google.com/bigquery/docs/reference/standard-sql/data-definition-language#creating_a_new_table_from_an_existing_table)
statement.


For the next example we first create a local copy of the data so that we have Write permissions:

In [140]:
%%bash

# Create a dataset in your project
bq mk --location eu my_london_bicycles_dataset

# Copy the public dataset to your project
bq cp bigquery-public-data:london_bicycles.cycle_hire my_london_bicycles_dataset.cycle_hire
bq cp bigquery-public-data:london_bicycles.cycle_stations my_london_bicycles_dataset.cycle_stations

BigQuery error in mk operation: Dataset 'dhodun1:my_london_bicycles_dataset'
already exists.
cp: replace dhodun1:my_london_bicycles_dataset.cycle_hire? (y/N) cp: replace dhodun1:my_london_bicycles_dataset.cycle_hire? (y/N) 
Got EOF; exiting. Is your input from a terminal?


CalledProcessError: Command 'b'\n# Create a dataset in your project\nbq mk --location eu my_london_bicycles_dataset\n\n# Copy the public dataset to your project\nbq cp bigquery-public-data:london_bicycles.cycle_hire my_london_bicycles_dataset.cycle_hire\nbq cp bigquery-public-data:london_bicycles.cycle_stations my_london_bicycles_dataset.cycle_stations\n'' returned non-zero exit status 1.

In [126]:
%%bash

# Examine your local table
bq show my_london_bicycles_dataset.cycle_hire

Table dhodun1:my_london_bicycles_dataset.cycle_hire

   Last modified                      Schema                     Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Labels  
 ----------------- -------------------------------------------- ------------ ------------- ------------ ------------------- ------------------ -------- 
  24 Nov 16:35:05   |- rental_id: integer (required)             24369201     2784394803                                                                
                    |- duration: integer                                                                                                                
                    |- bike_id: integer                                                                                                                 
                    |- end_date: timestamp                                                                                                              
                    |- end_st

__#TODO(you):__ Rewrite this `INSERT INTO` query so that it only executes one DML transaction

In [129]:
%%bigquery
-- Rows before

SELECT COUNT(*) FROM my_london_bicycles_dataset.cycle_hire

,f0_
0,24369202


In [134]:
%%bigquery
-- Query to be edited
INSERT INTO
  my_london_bicycles_dataset.cycle_hire
VALUES
  (47469109, 3180, 7054, '2015-09-03 12:45:00 UTC', 111, 'Park Lane, Hyde Park', '2015-09-03 11:52:00 UTC', 300, 'Serpentine Car Park, Hyde Park', NULL, NULL, NULL);
INSERT INTO
  my_london_bicycles_dataset.cycle_hire
VALUES
  (46915469, 7380, 3792, '2015-08-16 11:59:00 UTC', 407, 'Speakers\' Corner 1, Hyde Park', '2015-08-16 09:56:00 UTC', 407, 'Speakers\' Corner 1, Hyde Park', NULL, NULL, NULL);

""


In [ ]:
%%bigquery
-- Fixed query
INSERT INTO
  my_london_bicycles_dataset.cycle_hire
VALUES
  (47469109, 3180, 7054, '2015-09-03 12:45:00 UTC', 111, 'Park Lane, Hyde Park', '2015-09-03 11:52:00 UTC', 300, 'Serpentine Car Park, Hyde Park', NULL, NULL, NULL),
  (46915469, 7380, 3792, '2015-08-16 11:59:00 UTC', 407, 'Speakers\' Corner 1, Hyde Park', '2015-08-16 09:56:00 UTC', 407, 'Speakers\' Corner 1, Hyde Park', NULL, NULL, NULL);

In [133]:
%%bigquery
-- Rows after

SELECT COUNT(*) FROM my_london_bicycles_dataset.cycle_hire

,f0_
0,24369205


### The UPDATE statement

`UPDATE` statements in Teradata are similar to `UPDATE` statements in standard
SQL. The important differences are:

-   The order of the `SET` and `FROM` clauses is reversed.
-   Any
    [Teradata correlation names](https://docs.teradata.com/reader/huc7AEHyHSROUkrYABqNIg/k6fC7ozmhIZZXa315VjJAw)
    used as table aliases in the `UPDATE` must be removed.
-   In Standard SQL, each `UPDATE` statement must include the `WHERE` keyword,
    followed by a condition. To update all rows in the table, use `WHERE true`.

The following example shows an `UPDATE` statement from Teradata that uses
joins:

```sql
UPDATE t1
FROM t1, t2
SET
 b = t2.b
WHERE a = t2.a;
```

The equivalent statement in standard SQL is the following:

```sql
UPDATE t1
SET
 b = t2.b
FROM t2
WHERE a = t2.a;
```

The considerations from the previous section about executing large numbers of
DML statements in BigQuery also apply in this case. We recommend
using a single
[`MERGE`](https://cloud.google.com/bigquery/docs/reference/standard-sql/dml-syntax#merge_statement)
statement instead of multiple `UPDATE` statements.


__#TODO(you):__ Rewrite this UPDATE statement so that it executes in BigQuery

In [146]:
%%bigquery --verbose
-- Query will fail
UPDATE
  my_london_bicycles_dataset.cycle_hire
FROM my_london_bicycles_dataset.cycle_hire t1, `bigquery-public-data`.london_bicycles.cycle_hire t2 
SET
  bike_id = t2.bike_id 
WHERE
  t1.rental_id = t2.rental_id

Executing query with job ID: b4cedc82-5022-478b-b2dd-fbb060d5c4a7
Query executing: 0.29s


ERROR:
 400 Syntax error: Unexpected keyword FROM at [4:1]

(job ID: b4cedc82-5022-478b-b2dd-fbb060d5c4a7)

                                     -----Query Job SQL Follows-----                                     

    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |
   1:-- Query will fail
   2:UPDATE
   3:  my_london_bicycles_dataset.cycle_hire
   4:FROM my_london_bicycles_dataset.cycle_hire t1, `bigquery-public-data`.london_bicycles.cycle_hire t2 
   5:SET
   6:  bike_id = t2.bike_id 
   7:WHERE
   8:  t1.rental_id = t2.rental_id
    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |    .    |


In [143]:
%%bigquery --verbose
-- Query will succeed but take a bit of time
UPDATE
  my_london_bicycles_dataset.cycle_hire t1
SET
  bike_id = t2.bike_id
FROM
  `bigquery-public-data`.london_bicycles.cycle_hire t2
WHERE
  t1.rental_id = t2.rental_id

Executing query with job ID: 58889cf8-5eca-4993-b7cf-edb800462aae
Query executing: 47.41s
Query complete after 48.42s


""


### The DELETE statement

Standard SQL requires `DELETE` statements to have a `WHERE` clause. In
Teradata, `WHERE` clauses are
[optional in `DELETE` statements](https://docs.teradata.com/reader/huc7AEHyHSROUkrYABqNIg/z8eO9bdxtjFRveHdDwwYPQ)
if you're deleting all the rows in a table. (If specific rows are being deleted,
the Teradata DML also requires a `WHERE` clause.) During translation, any
missing `WHERE` clauses must be added to scripts. This change is necessary only
when all the rows in a table will be deleted.

For instance, the following statement in Teradata SQL deletes all the rows from
a table. The `ALL` clause is optional:

```sql
DELETE t1 ALL;
```

The translation into standard SQL is as follows:

```sql
DELETE FROM t1 WHERE TRUE;
```

__#TODO(you):__ Rewrite this UPDATE statement so that it executes in BigQuery

In [148]:
%%bigquery
--Query will fail

DELETE my_london_bicycles_dataset.cycle_hire ALL;

Executing query with job ID: ae44c518-de4f-403e-bda3-7102b7e829bc
Query executing: 0.25s


ERROR:
 400 Syntax error: Unexpected keyword ALL at [3:46]

(job ID: ae44c518-de4f-403e-bda3-7102b7e829bc)

           -----Query Job SQL Follows-----            

    |    .    |    .    |    .    |    .    |
   1:--Query will fail
   2:
   3:DELETE my_london_bicycles_dataset.cycle_hire ALL;
    |    .    |    .    |    .    |    .    |


In [149]:
%%bigquery
--Query will succeed

DELETE my_london_bicycles_dataset.cycle_hire WHERE TRUE;

""


## Stored Procedures